In [9]:
import garminconnect
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [10]:
from getpass import getpass

email = input("Enter Garmin Connect email address: ")
password = getpass("Enter Garmin Connect password: ")

garmin = garminconnect.Garmin(email, password)
garmin.login()
assert garmin.display_name

In [11]:
import os

GARTH_HOME = os.getenv("GARTH_HOME", "~/.garth")
garmin.garth.dump(GARTH_HOME)

In [12]:
activities_running = garmin.get_activities_by_date(startdate='2023-01-01', enddate="2024-12-31", activitytype='running')

In [13]:
# Transform data
df_activities_running = pd.json_normalize(activities_running, sep='_')
df_activities_running = df_activities_running.sort_values(by='startTimeLocal', ascending=True)
df_activities_running = df_activities_running.reset_index(drop=True)
df_activities_running['startTimeLocal'] = pd.to_datetime(df_activities_running['startTimeLocal'])
df_activities_running['endTimeLocal'] = df_activities_running.startTimeLocal + pd.to_timedelta(df_activities_running.duration, unit='sec')
df_activities_running[['startTimeLocal', 'endTimeLocal', 'duration']]


,startTimeLocal,endTimeLocal,duration
0,2023-02-19 12:41:18,2023-02-19 13:10:53.308959961,1775.308960
1,2023-03-05 10:08:42,2023-03-05 10:44:36.283935547,2154.283936
2,2023-03-26 10:02:41,2023-03-26 10:07:03.683013916,262.683014
3,2023-04-14 17:20:33,2023-04-14 17:23:45.457992554,192.457993
4,2023-04-30 16:57:40,2023-04-30 17:26:53.458984375,1753.458984
...,...,...,...
153,2024-03-20 16:20:15,2024-03-20 17:09:50.792968750,2975.792969
154,2024-03-23 12:27:56,2024-03-23 13:24:27.310058594,3391.310059
155,2024-03-25 08:45:26,2024-03-25 09:49:33.631103516,3847.631104
156,2024-03-27 17:18:31,2024-03-27 17:41:28.796020508,1377.796021


In [14]:
fig = px.scatter(
    data_frame=df_activities_running,
    x='startTimeLocal',
    y='distance',
    color='averageSpeed',
    )
fig.show()

In [15]:
# Define bins & labels
bins = [0, 5_000, 10_000, 21_100, 42_100, 100_000]
labels = ['0-5km', '≥ 5km', '≥ 10km', 'HM (≥21.1km)', 'Marathon (≥42.1km)']

# Cut distances into intervals and count the occurrences
df_activities_running['distance_interval'] = pd.cut(df_activities_running['distance'], bins=bins, labels=labels, right=False)
counts = df_activities_running['distance_interval'].value_counts().reindex(labels)

# Create a Plotly bar chart
fig = go.Figure(data=[go.Bar(x=counts.index, y=counts.values)])

# Add text annotations
for i, count in enumerate(counts):
    fig.add_annotation(
        x=counts.index[i],
        y=count + 4,  # Adjust the position of the annotation above the bar
        text=str(count),
        showarrow=False
    )

# Update layout
fig.update_layout(
    title='Number of Runs in Distance Intervals',
    xaxis=dict(title='Distance Intervals'),
    yaxis=dict(title='Number of Runs')
)

# Show the plot
fig.show()